In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
pd.__version__ # get the version of pandas

u'0.18.0'

In [3]:
def apply_2(df_train,df_test,f,args):
    return(f(df_train,args),f(df_test,args))

def apply_3(df_train,df_valid,df_test,f,args):
    return(f(df_train,args),f(df_valid,args),f(df_test,args))


def set_columns(df,args):
    df.columns = args
    return(df)

def del_columns(df,args):
    return(df.drop(args,axis=1))

def assign(df,args): #['colname',g]
    colname=args[0]
    g=args[1]
    df[colname]=df['C1'].apply(g)
    return(df)

In [4]:
df_raw=pd.read_csv('/Users/francois-guillaume.rideau/Documents/Kaggle/Kaggle-SF/train.csv')
df_test_raw=pd.read_csv('/Users/francois-guillaume.rideau/Documents/Kaggle/Kaggle-SF/test.csv')

In [5]:
print(df_raw.shape)
print(df_test_raw.shape)
Nobs_train_raw = df_raw.shape[0]
Nobs_train_raw
df_raw['Id']=range(Nobs_train_raw)

(878049, 9)
(884262, 7)


In [6]:
minidf=df_raw.loc[0:5,:]
minidf
minidf_test=df_test_raw.loc[0:5,:]
minidf_test

,Id,Dates,DayOfWeek,PdDistrict,Address,X,Y
0,0,2015-05-10 23:59:00,Sunday,BAYVIEW,2000 Block of THOMAS AV,-122.399588,37.735051
1,1,2015-05-10 23:51:00,Sunday,BAYVIEW,3RD ST / REVERE AV,-122.391523,37.732432
2,2,2015-05-10 23:50:00,Sunday,NORTHERN,2000 Block of GOUGH ST,-122.426002,37.792212
3,3,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412
4,4,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412
5,5,2015-05-10 23:40:00,Sunday,TARAVAL,BROAD ST / CAPITOL AV,-122.459024,37.713172


In [7]:
df_raw.describe() # shows one value for Y which is totally stupid

,X,Y,Id
count,878049.000000,878049.000000,878049.000000
mean,-122.422616,37.771020,439024.000000
std,0.030354,0.456893,253471.057593
min,-122.513642,37.707879,0.000000
25%,-122.432952,37.752427,219512.000000
50%,-122.416420,37.775421,439024.000000
75%,-122.406959,37.784369,658536.000000
max,-120.500000,90.000000,878048.000000


In [8]:
df_raw[df_raw['Y']>38] # note that X is -120.5 for all those obs. We still have PdDistrict and Address...67 such OBS

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,Id
660485,2005-12-30 17:00:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Friday,TENDERLOIN,NONE,5THSTNORTH ST / OFARRELL ST,-120.5,90.0,660485
660711,2005-12-30 00:34:00,ASSAULT,INFLICT INJURY ON COHABITEE,Friday,BAYVIEW,"ARREST, BOOKED",JAMESLICKFREEWAY HY / SILVER AV,-120.5,90.0,660711
660712,2005-12-30 00:34:00,ASSAULT,AGGRAVATED ASSAULT WITH BODILY FORCE,Friday,BAYVIEW,"ARREST, BOOKED",JAMESLICKFREEWAY HY / SILVER AV,-120.5,90.0,660712
661106,2005-12-29 00:07:00,NON-CRIMINAL,"AIDED CASE, MENTAL DISTURBED",Thursday,TENDERLOIN,PSYCHOPATHIC CASE,5THSTNORTH ST / EDDY ST,-120.5,90.0,661106
666430,2005-11-30 11:25:00,OTHER OFFENSES,TRAFFIC VIOLATION,Wednesday,TENDERLOIN,"ARREST, CITED",5THSTNORTH ST / ELLIS ST,-120.5,90.0,666430
667042,2005-11-28 16:04:00,TRESPASS,TRESPASSING,Monday,TENDERLOIN,"ARREST, BOOKED",ELLIS ST / 5THSTNORTH ST,-120.5,90.0,667042
669946,2005-11-14 09:20:00,OTHER OFFENSES,"DRIVERS LICENSE, SUSPENDED OR REVOKED",Monday,BAYVIEW,"ARREST, BOOKED",YOSEMITE AV / WILLIAMS AV,-120.5,90.0,669946
671709,2005-11-02 20:30:00,OTHER OFFENSES,MISCELLANEOUS INVESTIGATION,Wednesday,CENTRAL,NONE,BRENHAM PL / WASHINGTON ST,-120.5,90.0,671709
673078,2005-10-23 20:00:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Sunday,RICHMOND,NONE,AVENUE OF THE PALMS / GEARY BL,-120.5,90.0,673078
673114,2005-10-23 18:11:00,WARRANTS,ENROUTE TO OUTSIDE JURISDICTION,Sunday,TARAVAL,"ARREST, BOOKED",STCHARLES AV / 19TH AV,-120.5,90.0,673114


In [9]:
df_raw.isnull().values.any() # no missing values

False

In [10]:
df_test_raw.describe() # shows one value for Y which is totally stupid

,Id,X,Y
count,884262.000000,884262.000000,884262.000000
mean,442130.500000,-122.422693,37.771476
std,255264.596205,0.030985,0.484824
min,0.000000,-122.513642,37.707879
25%,221065.250000,-122.433069,37.752374
50%,442130.500000,-122.416517,37.775421
75%,663195.750000,-122.406959,37.784353
max,884261.000000,-120.500000,90.000000


In [11]:
df_test_raw[df_test_raw['Y']>38] # note that X is -120.5 for all those obs. We still have PdDistrict and Address...76 such OBS

,Id,Dates,DayOfWeek,PdDistrict,Address,X,Y
664733,664733,2005-12-20 14:57:00,Tuesday,TENDERLOIN,7THSTNORTH ST / MCALLISTER ST,-120.5,90.0
664734,664734,2005-12-20 14:57:00,Tuesday,TENDERLOIN,7THSTNORTH ST / MCALLISTER ST,-120.5,90.0
664826,664826,2005-12-20 07:36:00,Tuesday,RICHMOND,GEARY BL / AVENUE OF THE PALMS,-120.5,90.0
664895,664895,2005-12-19 23:08:00,Monday,TENDERLOIN,EDDY ST / 5THSTNORTH ST,-120.5,90.0
667226,667226,2005-12-06 09:40:00,Tuesday,RICHMOND,GEARY BL / AVENUE OF THE PALMS,-120.5,90.0
671467,671467,2005-11-10 16:30:00,Thursday,TENDERLOIN,MCALLISTER ST / 7THSTNORTH ST,-120.5,90.0
672453,672453,2005-11-08 06:30:00,Tuesday,CENTRAL,FRONT ST / THE EMBARCADERONORTH ST,-120.5,90.0
675611,675611,2005-10-24 00:03:00,Monday,TENDERLOIN,OFARRELL ST / 5THSTNORTH ST,-120.5,90.0
675858,675858,2005-10-16 10:27:00,Sunday,TENDERLOIN,EDDY ST / 5THSTNORTH ST,-120.5,90.0
676528,676528,2005-10-14 15:49:00,Friday,TENDERLOIN,EDDY ST / 5THSTNORTH ST,-120.5,90.0


In [12]:
df_test_raw.isnull().values.any() # no missing values

False

In [13]:
# converting columns to the proper format (training set)

df_raw['Dates']=pd.to_datetime(df_raw['Dates'])

def is_categorical_a(array_like):
    return array_like.dtype.name == 'category'

for col in ['Category', 'DayOfWeek', 'PdDistrict', 'Resolution']:
    df_raw[col] = df_raw[col].astype('category')
    
df_raw.dtypes

# converting columns to the proper format (test set)

df_test_raw['Dates']=pd.to_datetime(df_test_raw['Dates'])

for col in ['DayOfWeek', 'PdDistrict']:
    df_test_raw[col] = df_test_raw[col].astype('category')
    
df_test_raw.dtypes
df_t = df_test_raw

In [14]:
# create a clean dataframe for the training test

df=df_raw[df_raw['Y']<=38]

# explore the dataframe
df_raw['PdDistrict'].describe()
df_raw.dtypes

Dates         datetime64[ns]
Category            category
Descript              object
DayOfWeek           category
PdDistrict          category
Resolution          category
Address               object
X                    float64
Y                    float64
Id                     int64
dtype: object

In [15]:
df.loc[:,'Hour']=df['Dates'].dt.hour
df.loc[:,'Month']=df['Dates'].dt.month # normalization
type(df.loc)
# df['Hour']

df_t.loc[:,'Hour']=df_t['Dates'].dt.hour
df_t.loc[:,'Month']=df_t['Dates'].dt.month# normalization
type(df_t.loc)
df

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/pandas/core/indexing.py:284: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/pandas/core/indexing.py:461: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,Id,Hour,Month
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,0,23,5
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,1,23,5
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414,2,23,5
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873,3,23,5
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541,4,23,5
5,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM UNLOCKED AUTO,Wednesday,INGLESIDE,NONE,0 Block of TEDDY AV,-122.403252,37.713431,5,23,5
6,2015-05-13 23:30:00,VEHICLE THEFT,STOLEN AUTOMOBILE,Wednesday,INGLESIDE,NONE,AVALON AV / PERU AV,-122.423327,37.725138,6,23,5
7,2015-05-13 23:30:00,VEHICLE THEFT,STOLEN AUTOMOBILE,Wednesday,BAYVIEW,NONE,KIRKWOOD AV / DONAHUE ST,-122.371274,37.727564,7,23,5
8,2015-05-13 23:00:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,RICHMOND,NONE,600 Block of 47TH AV,-122.508194,37.776601,8,23,5
9,2015-05-13 23:00:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,CENTRAL,NONE,JEFFERSON ST / LEAVENWORTH ST,-122.419088,37.807802,9,23,5


In [16]:
# example how to make dummy variables
s= pd.Series(["Wednesday","Tuesday","Friday","Thursday","Saturday","Sunday","Monday"], dtype="category")
s = s.cat.reorder_categories(["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"], ordered=True)
pd.get_dummies(s, drop_first=True)

df.loc[:,'DayOfWeek'] = df['DayOfWeek'].cat.reorder_categories(["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"], ordered=True)
x=pd.get_dummies(df['DayOfWeek'], drop_first=True)
y=pd.get_dummies(df['PdDistrict'], drop_first=True)

In [17]:
# df['Hour'].describe() # 0 to 23

def hourslice2 (hour):
    if   (hour==0)  or (hour==23) : return ("23H - 0H")
    elif (hour==1)  or (hour==2)  : return (" 1H - 2H")
    elif (hour==3)  or (hour==4)  : return (" 3H - 4H")
    elif (hour==5)  or (hour==6)  : return (" 5H - 6H")
    elif (hour==7)  or (hour==8)  : return (" 7H - 8H")
    elif (hour==9)  or (hour==10) : return (" 9H - 10H")
    elif (hour==11) or (hour==12) : return ("11H - 12H")
    elif (hour==13) or (hour==14) : return ("13H - 14H")
    elif (hour==15) or (hour==16) : return ("15H - 16H")
    elif (hour==17) or (hour==18) : return ("17H - 18H")
    elif (hour==19) or (hour==20) : return ("19H - 20H")
    else                          : return ("21H - 22H")
    
df.loc[:,'hourslice2']=df.loc[:,'Hour'].apply(hourslice2)   
z = pd.get_dummies(df['hourslice2'], drop_first=True)

In [18]:
def season(month):
    if   (month==1) or (month==2) or (month==12) : return("winter")
    elif (month==3) or (month==4) or (month==5)  : return("spring")
    elif (month==6) or (month==7) or (month==8)  : return("summer")
    else                                         : return("autumn")
df.loc[:,'season']=df.loc[:,'Month'].apply(season)   
t = pd.get_dummies(df['season'], drop_first=True)   

df.loc[:,'Month']=df.loc[:,'Month']/12

In [19]:
df = pd.concat([df, x,y,z,t], axis=1)

In [20]:
def myfun(y): return(y.value // 10**9)
    
df.loc[:,'Date_sec']=df['Dates'].apply(myfun)

In [21]:
df_t.loc[:,'DayOfWeek'] = df_t['DayOfWeek'].cat.reorder_categories(["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"], ordered=True)
x=pd.get_dummies(df_t['DayOfWeek'], drop_first=True)
y=pd.get_dummies(df_t['PdDistrict'], drop_first=True)
df_t.loc[:,'hourslice2']=df_t.loc[:,'Hour'].apply(hourslice2)   
z = pd.get_dummies(df_t['hourslice2'], drop_first=True)
df_t.loc[:,'season']=df_t.loc[:,'Month'].apply(season)   
t = pd.get_dummies(df_t['season'], drop_first=True)   
df_t.loc[:,'Month']=df_t.loc[:,'Month']/12


df_t = pd.concat([df_t, x,y,z,t], axis=1)

df_t.loc[:,'Date_sec']=df_t['Dates'].apply(myfun)

In [22]:
import holidays
from datetime import date
us_holidays = holidays.UnitedStates()  # or holidays.US()

def isholiday(d) :
    return(int(d in us_holidays))
# example
int(isholiday('2001-01-01 23:53:00'))

df['Holiday']=df['Dates'].apply(isholiday)
df_t['Holiday']=df_t['Dates'].apply(isholiday)



In [23]:
df_t.head()

,Id,Dates,DayOfWeek,PdDistrict,Address,X,Y,Hour,Month,hourslice2,...,15H - 16H,17H - 18H,19H - 20H,21H - 22H,23H - 0H,spring,summer,winter,Date_sec,Holiday
0,0,2015-05-10 23:59:00,Sunday,BAYVIEW,2000 Block of THOMAS AV,-122.399588,37.735051,23,0.416667,23H - 0H,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1431302340,0
1,1,2015-05-10 23:51:00,Sunday,BAYVIEW,3RD ST / REVERE AV,-122.391523,37.732432,23,0.416667,23H - 0H,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1431301860,0
2,2,2015-05-10 23:50:00,Sunday,NORTHERN,2000 Block of GOUGH ST,-122.426002,37.792212,23,0.416667,23H - 0H,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1431301800,0
3,3,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412,23,0.416667,23H - 0H,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1431301500,0
4,4,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412,23,0.416667,23H - 0H,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1431301500,0


In [24]:
# http://sanfranciscopolice.org/police-district-maps
# http://www.latlong.net/convert-address-to-lat-long.html

PoliceStations_raw=pd.DataFrame(
[['BAYVIEW'   ,-122.397981,37.729732],
 ['CENTRAL'   ,-122.409919,37.798732],
 ['INGLESIDE' ,-122.446215,37.724676],
 ['MISSION'   ,-122.422005,37.762849],
 ['NORTHERN'  ,-122.432467,37.780186],
 ['PARK'      ,-122.455287,37.767797],
 ['RICHMOND'  ,-122.464467,37.779928],
 ['SOUTHERN'  ,-122.389412,37.772380],
 ['TARAVAL'   ,-122.481500,37.743733],
 ['TENDERLOIN',-122.412899,37.783674]],columns=['PdDistrict','XX','YY'])
PoliceStations_raw

,PdDistrict,XX,YY
0,BAYVIEW,-122.397981,37.729732
1,CENTRAL,-122.409919,37.798732
2,INGLESIDE,-122.446215,37.724676
3,MISSION,-122.422005,37.762849
4,NORTHERN,-122.432467,37.780186
5,PARK,-122.455287,37.767797
6,RICHMOND,-122.464467,37.779928
7,SOUTHERN,-122.389412,37.772380
8,TARAVAL,-122.481500,37.743733
9,TENDERLOIN,-122.412899,37.783674


In [25]:
df  =pd.merge(df  ,PoliceStations_raw,on='PdDistrict',how='left')
df_t=pd.merge(df_t,PoliceStations_raw,on='PdDistrict',how='left')

#df['Pd_Dist']=( (df['X']-df['XX']) **2 + (df['Y']-df['YY']) ** 2)
df.head()
# be careful pd.merge is resetting dtypes !!!!!!!
df.dtypes
df['Category'] = df_raw['Category'].astype('category')
df.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,Id,...,19H - 20H,21H - 22H,23H - 0H,spring,summer,winter,Date_sec,Holiday,XX,YY
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,0,...,0.0,0.0,1.0,1.0,0.0,0.0,1431561180,0,-122.432467,37.780186
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,1,...,0.0,0.0,1.0,1.0,0.0,0.0,1431561180,0,-122.432467,37.780186
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414,2,...,0.0,0.0,1.0,1.0,0.0,0.0,1431559980,0,-122.432467,37.780186
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873,3,...,0.0,0.0,1.0,1.0,0.0,0.0,1431559800,0,-122.432467,37.780186
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541,4,...,0.0,0.0,1.0,1.0,0.0,0.0,1431559800,0,-122.455287,37.767797


In [26]:
df['Pd_Dist']=( (df['X']-df['XX']) **2 + (df['Y']-df['YY']) ** 2)
df['Pd_Dist']=np.sqrt(df['Pd_Dist'])

df_t['Pd_Dist']=( (df_t['X']-df_t['XX']) **2 + (df_t['Y']-df_t['YY']) ** 2)
df_t['Pd_Dist']=np.sqrt(df_t['Pd_Dist'])
df.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,Id,...,21H - 22H,23H - 0H,spring,summer,winter,Date_sec,Holiday,XX,YY,Pd_Dist
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,0,...,0.0,1.0,1.0,0.0,0.0,1431561180,0,-122.432467,37.780186,0.008629
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,1,...,0.0,1.0,1.0,0.0,0.0,1431561180,0,-122.432467,37.780186,0.008629
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414,2,...,0.0,1.0,1.0,0.0,0.0,1431559980,0,-122.432467,37.780186,0.021791
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873,3,...,0.0,1.0,1.0,0.0,0.0,1431559800,0,-122.432467,37.780186,0.021398
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541,4,...,0.0,1.0,1.0,0.0,0.0,1431559800,0,-122.455287,37.767797,0.016968


In [27]:
# normalize test data with training data scaling !!!!! very important !!!! (easy mistake)

def norm_01(df1,df2):
    return( ( df1 - df1.min()) / (df1.max() - df1.min()) , ( df2 - df1.min()) / (df1.max() - df1.min()) ) 

In [28]:
df['Xnorm'],df_t['Xnorm']=norm_01(df['X'],df_t['X'])
df['Ynorm'],df_t['Ynorm']=norm_01(df['Y'],df_t['Y'])
df['Hournorm'],df_t['Hournorm']=norm_01(df['Hour'],df_t['Hour'])
df['Date_secnorm'],df_t['Date_secnorm']=norm_01(df['Date_sec'],df_t['Date_sec'])
df['Pd_Distnorm'],df_t['Pd_Distnorm']= norm_01(df['Pd_Dist'],df_t['Pd_Dist'])

In [29]:
# Adding Address Features
df_a=df.loc[:,'Address']
df_t_a = df_t.loc[:,'Address']


df['AV']=df_a.apply(lambda x: int(' AV' in x))
df['BL']=df_a.apply(lambda x: int(' BL' in x))
df['CR']=df_a.apply(lambda x: int(' CR' in x))
df['CT']=df_a.apply(lambda x: int(' CT' in x))
df['DR']=df_a.apply(lambda x: int(' DR' in x))
df['BL']=df_a.apply(lambda x: int(' BL' in x))
df['LN']=df_a.apply(lambda x: int(' LN' in x))
df['PL']=df_a.apply(lambda x: int(' PL' in x))
df['PZ']=df_a.apply(lambda x: int(' PZ' in x))
df['RD']=df_a.apply(lambda x: int(' RD' in x))
df['TR']=df_a.apply(lambda x: int(' TR' in x))
df['HWY']=df_a.apply(lambda x: int((' HY' in x) or ('HWY' in x)) )
df['WAY']=df_a.apply(lambda x: int((' WY' in x) or ('WAY' in x)) )
#i took out 'ST' because everything not above contains ST

df['IsIntersect'] = df_a.apply(lambda x: int('/' in x))

# same for test set

df_t['AV'] =df_t_a.apply(lambda x: int(' AV' in x))
df_t['BL'] =df_t_a.apply(lambda x: int(' BL' in x))
df_t['CR'] =df_t_a.apply(lambda x: int(' CR' in x))
df_t['CT'] =df_t_a.apply(lambda x: int(' CT' in x))
df_t['DR'] =df_t_a.apply(lambda x: int(' DR' in x))
df_t['BL'] =df_t_a.apply(lambda x: int(' BL' in x))
df_t['LN'] =df_t_a.apply(lambda x: int(' LN' in x))
df_t['PL'] =df_t_a.apply(lambda x: int(' PL' in x))
df_t['PZ'] =df_t_a.apply(lambda x: int(' PZ' in x))
df_t['RD'] =df_t_a.apply(lambda x: int(' RD' in x))
df_t['TR'] =df_t_a.apply(lambda x: int(' TR' in x))
df_t['HWY']=df_t_a.apply(lambda x: int((' HY' in x) or ('HWY' in x)) )
df_t['WAY']=df_t_a.apply(lambda x: int((' WY' in x) or ('WAY' in x)) )
#i took out 'ST' because everything not above contains ST

df_t['IsIntersect'] = df_t_a.apply(lambda x: int('/' in x))
# 'AV','BL','CR','CT','DR','BL','LN','PL','PZ','RD','TR','HWY','WAY','IsIntersect'

In [30]:
NObs = df.shape[0]
NObs//10
87798%128

import random as random

random.seed(1234)

rows = random.sample(df.index, NObs//10)
# rows1 = rows[:len(rows)//2]
# rows2 = rows[len(rows)//2:]
# validdata = df.ix[rows1]
# testdata  = df.ix[rows2]

validdata =df.ix[rows]
# validdata =validdata.drop('Id',axis=1)

traindata = df.drop(rows)
#traindata=traindata.drop('Id',axis=1)

print(traindata.shape)
print(validdata.shape)

testdata = df_t

(790184, 66)
(87798, 66)


In [31]:
df_a_train = traindata.loc[:,'Address']
df_a_valid = validdata.loc[:,'Address']

address_train=pd.DataFrame(sorted(df_a_train.unique()),columns=['Address'])
address_valid=pd.DataFrame(sorted(df_a_valid.unique()),columns=['Address'])
address_test=pd.DataFrame(sorted(df_t_a.unique()),columns=['Address'])
print('number of addresses in train set %d' % len(address_train))
print('number of addresses in valid set %d' % len(address_valid))
print('number of addresses in test  set %d' % len(address_test))
# categories=sorted(df["Category"].unique())
C_counts=traindata.groupby(["Category"]).size()
N_obs_train=len(traindata)

print(sum(C_counts))
print(len(traindata))
# A_C_counts=df.groupby(["Address","Category"]).size()

A_counts=traindata.groupby(["Address"]).size()
A_counts=pd.DataFrame(A_counts)
A_counts.columns=['count']
A_counts.head()


number of addresses in train set 22941
number of addresses in valid set 14956
number of addresses in test  set 23184
790184
790184


,count
Address,
0 Block of 10TH AV,5
0 Block of 10TH ST,42
0 Block of 11TH ST,43
0 Block of 12TH AV,7
0 Block of 12TH ST,382


In [32]:
check_address_valid=pd.merge(address_valid,A_counts,left_on='Address',right_index=True,how='left',indicator=True)
check_address_test =pd.merge(address_test ,A_counts,left_on='Address',right_index=True,how='left',indicator=True)
print('number of addresses in valid set not present in the train set %d' % len(check_address_valid[check_address_valid['_merge']=='left_only']))
print('number of addresses in test  set not present in the train set %d' % len(check_address_test [check_address_test ['_merge']=='left_only']))
# 1563 new addresses in the test set, how many observations is that ?
check_address_test

number of addresses in valid set not present in the train set 250
number of addresses in test  set not present in the train set 1716


,Address,count,_merge
0,0 Block of 10TH ST,42.0,both
1,0 Block of 11TH ST,43.0,both
2,0 Block of 12TH AV,7.0,both
3,0 Block of 12TH ST,382.0,both
4,0 Block of 13TH ST,41.0,both
5,0 Block of 14TH AV,6.0,both
6,0 Block of 14TH ST,52.0,both
7,0 Block of 15TH AV,5.0,both
8,0 Block of 15TH ST,7.0,both
9,0 Block of 16TH AV,5.0,both


In [33]:
address_test.head()

,Address
0,0 Block of 10TH ST
1,0 Block of 11TH ST
2,0 Block of 12TH AV
3,0 Block of 12TH ST
4,0 Block of 13TH ST


In [34]:
df1 = pd.merge(traindata,A_counts,left_on='Address',how='left',right_index=True)
df1['one']=1

table = pd.pivot_table(df1, values='one',index=['Address'],
                 columns=['Category'], fill_value=0, aggfunc=np.sum)
Total=table.sum(axis=1)

def myfun(v):
    w=v.astype(float)
    return(np.divide(w,Total))
table=table.apply(myfun)
table
table.columns=["proba_"+str(x) for x in range(len(table.columns))]

df2=pd.merge(df1,table,left_on='Address',how='left',right_index=True).drop(['one'],axis=1)

In [35]:
table.loc['Default',:]=np.array(C_counts/N_obs_train)
print(sum(table.loc['Default',:]))
table.tail()

1.0


,proba_0,proba_1,proba_2,proba_3,proba_4,proba_5,proba_6,proba_7,proba_8,proba_9,...,proba_29,proba_30,proba_31,proba_32,proba_33,proba_34,proba_35,proba_36,proba_37,proba_38
Address,,,,,,,,,,,,,,,,,,,,,
ZOE ST / BRANNAN ST,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000
ZOE ST / BRYANT ST,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.142857,0.000000,0.000000
ZOE ST / FREELON ST,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000
ZOE ST / WELSH ST,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Default,0.00173,0.08748,0.000461,0.000337,0.041871,0.004906,0.002537,0.061507,0.004871,0.001328,...,0.000168,0.005172,0.000577,0.035849,0.000004,0.008355,0.050935,0.061231,0.047971,0.009717


In [36]:
df_v_1 = pd.merge(validdata,check_address_valid,on='Address',how='left')
df_v_1['Address'][df_v_1['_merge']=='left_only']='Default'
# number of obs in the valid set with Default address
print(sum((df_v_1['Address']=='Default').astype(int)))


df_t_1 = pd.merge(testdata,check_address_test,on='Address',how='left')
df_t_1['Address'][df_t_1['_merge']=='left_only']='Default'
# number of obs in the test set with Default address
print(sum((df_t_1['Address']=='Default').astype(int)))

df_t_1

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


274


/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


3681


,Id,Dates,DayOfWeek,PdDistrict,Address,X,Y,Hour,Month,hourslice2,...,LN,PL,PZ,RD,TR,HWY,WAY,IsIntersect,count,_merge
0,0,2015-05-10 23:59:00,Sunday,BAYVIEW,2000 Block of THOMAS AV,-122.399588,37.735051,23,0.416667,23H - 0H,...,0,0,0,0,0,0,0,0,64.0,both
1,1,2015-05-10 23:51:00,Sunday,BAYVIEW,3RD ST / REVERE AV,-122.391523,37.732432,23,0.416667,23H - 0H,...,0,0,0,0,0,0,0,1,188.0,both
2,2,2015-05-10 23:50:00,Sunday,NORTHERN,2000 Block of GOUGH ST,-122.426002,37.792212,23,0.416667,23H - 0H,...,0,0,0,0,0,0,0,0,20.0,both
3,3,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412,23,0.416667,23H - 0H,...,0,0,0,0,0,0,0,0,443.0,both
4,4,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412,23,0.416667,23H - 0H,...,0,0,0,0,0,0,0,0,443.0,both
5,5,2015-05-10 23:40:00,Sunday,TARAVAL,BROAD ST / CAPITOL AV,-122.459024,37.713172,23,0.416667,23H - 0H,...,0,0,0,0,0,0,0,1,63.0,both
6,6,2015-05-10 23:30:00,Sunday,INGLESIDE,100 Block of CHENERY ST,-122.425616,37.739351,23,0.416667,23H - 0H,...,0,0,0,0,0,0,0,0,36.0,both
7,7,2015-05-10 23:30:00,Sunday,INGLESIDE,200 Block of BANKS ST,-122.412652,37.739750,23,0.416667,23H - 0H,...,0,0,0,0,0,0,0,0,14.0,both
8,8,2015-05-10 23:10:00,Sunday,MISSION,2900 Block of 16TH ST,-122.418700,37.765165,23,0.416667,23H - 0H,...,0,0,0,0,0,0,0,0,299.0,both
9,9,2015-05-10 23:10:00,Sunday,CENTRAL,TAYLOR ST / GREEN ST,-122.413935,37.798886,23,0.416667,23H - 0H,...,0,0,0,0,0,0,0,1,22.0,both


In [37]:
df_v_2=pd.merge(df_v_1,table,left_on='Address',how='left',right_index=True)
# print(df_v_2['count'].loc[0])

df_t_2=pd.merge(df_t_1,table,left_on='Address',how='left',right_index=True)
print(df_t_2['count'].loc[0])
print(A_counts.loc['2000 Block of THOMAS AV'])

# set count to 1 for addresses which are present in test set, but not in the training set

# xxx=df_t_2[df_t_2['Address']=='Default']
# df_t_2['count'][df_t_2['Address']=='Default']
df_v_2['count'][df_v_2['Address']=='Default']=1 
df_t_2['count'][df_t_2['Address']=='Default']=1 

df2   ['count']=df2.loc[:,'count']/N_obs_train
df_v_2['count']=df_v_2.loc[:,'count']/N_obs_train
df_t_2['count']=df_t_2.loc [:,'count']/N_obs_train

df_t_2.head()

64.0
count    64
Name: 2000 Block of THOMAS AV, dtype: int64


/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Id,Dates,DayOfWeek,PdDistrict,Address,X,Y,Hour,Month,hourslice2,...,proba_29,proba_30,proba_31,proba_32,proba_33,proba_34,proba_35,proba_36,proba_37,proba_38
0,0,2015-05-10 23:59:00,Sunday,BAYVIEW,2000 Block of THOMAS AV,-122.399588,37.735051,23,0.416667,23H - 0H,...,0.0,0.000000,0.000000,0.046875,0.0,0.000000,0.046875,0.046875,0.078125,0.031250
1,1,2015-05-10 23:51:00,Sunday,BAYVIEW,3RD ST / REVERE AV,-122.391523,37.732432,23,0.416667,23H - 0H,...,0.0,0.000000,0.000000,0.042553,0.0,0.000000,0.042553,0.063830,0.085106,0.010638
2,2,2015-05-10 23:50:00,Sunday,NORTHERN,2000 Block of GOUGH ST,-122.426002,37.792212,23,0.416667,23H - 0H,...,0.0,0.000000,0.000000,0.050000,0.0,0.000000,0.100000,0.200000,0.000000,0.000000
3,3,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412,23,0.416667,23H - 0H,...,0.0,0.002257,0.002257,0.047404,0.0,0.009029,0.067720,0.031603,0.045147,0.022573
4,4,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412,23,0.416667,23H - 0H,...,0.0,0.002257,0.002257,0.047404,0.0,0.009029,0.067720,0.031603,0.045147,0.022573


In [38]:
list_col_all = ['Id','Dates','Date_sec','Hour','season','PdDistrict','Address','X','Y','XX','YY','hourslice2','DayOfWeek']
#list_col_all = ['Id','Dates','Date_sec','Hour','season','PdDistrict','Address','X','Y','XX','YY','hourslice2','DayOfWeek','Pd_Dist',
#               'proba_0' ,'proba_1' ,'proba_2' ,'proba_3' ,'proba_4' ,'proba_5' ,'proba_6' ,'proba_7' ,'proba_8' ,'proba_9',
#               'proba_10','proba_11','proba_12','proba_13','proba_14','proba_15','proba_16','proba_17','proba_18','proba_19',
#               'proba_20','proba_21','proba_22','proba_23','proba_24','proba_25','proba_26','proba_27','proba_28','proba_29',
#               'proba_30','proba_31','proba_32','proba_33','proba_34','proba_35','proba_36','proba_37','proba_38',
#               'AV','BL','CR','CT','DR','BL','LN','PL','PZ','RD','TR','HWY','WAY','IsIntersect'               
#             ]

list_col_tr_v = ['Descript','Resolution']
list_col_v_te = ['_merge']

traindata,validdata,testdata=apply_3(df2,df_v_2,df_t_2,del_columns,list_col_all)
traindata,validdata=apply_2(traindata,validdata,del_columns,list_col_tr_v)
validdata,testdata=apply_2(validdata,testdata,del_columns,list_col_v_te)
testdata.tail()
print(len(testdata.columns))
print(len(validdata.columns))
print(len(traindata.columns))
# the 1 difference in column length is because train set includes Category.

90
91
91


In [ ]:
# execute this instead of the previous block if using the whole set for training
traindata=df
print(traindata.shape)

In [39]:
num_variables = traindata.shape[1]-1 # substract 1 because we will drop the target variable later
num_variables

90

In [40]:
num_Category=df['Category'].cat.categories.shape[0]
num_Category

39

In [41]:
Categories=np.array(traindata.loc[:,'Category'].cat.categories)
Categories

array(['ARSON', 'ASSAULT', 'BAD CHECKS', 'BRIBERY', 'BURGLARY',
       'DISORDERLY CONDUCT', 'DRIVING UNDER THE INFLUENCE',
       'DRUG/NARCOTIC', 'DRUNKENNESS', 'EMBEZZLEMENT', 'EXTORTION',
       'FAMILY OFFENSES', 'FORGERY/COUNTERFEITING', 'FRAUD', 'GAMBLING',
       'KIDNAPPING', 'LARCENY/THEFT', 'LIQUOR LAWS', 'LOITERING',
       'MISSING PERSON', 'NON-CRIMINAL', 'OTHER OFFENSES',
       'PORNOGRAPHY/OBSCENE MAT', 'PROSTITUTION', 'RECOVERED VEHICLE',
       'ROBBERY', 'RUNAWAY', 'SECONDARY CODES', 'SEX OFFENSES FORCIBLE',
       'SEX OFFENSES NON FORCIBLE', 'STOLEN PROPERTY', 'SUICIDE',
       'SUSPICIOUS OCC', 'TREA', 'TRESPASS', 'VANDALISM', 'VEHICLE THEFT',
       'WARRANTS', 'WEAPON LAWS'], dtype=object)

In [ ]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [ ]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

def baseline_logloss(hotmatrix): #input is a hot-encoded sparse matrix, one 1 only per row, rest is 0
    nobs = len(hotmatrix)
    temp = hotmatrix.sum(axis=0)/nobs
    return(-sum(np.multiply(temp,np.log(temp))))

In [ ]:
# prepare data for TensorFlow

num_var = num_variables   # 89
num_labels = num_Category # 39 categories

train_dataset = np.float32(np.array(traindata.drop(['Category'],axis=1)))
train_labels = traindata.loc[:,'Category']
train_labels= np.array(pd.get_dummies(train_labels.cat.rename_categories(np.arange(0,num_labels)) ))

valid_dataset = np.float32(np.array(validdata.drop(['Category'],axis=1)))
valid_labels = validdata.loc[:,'Category'].astype('category')
valid_labels= np.array(pd.get_dummies(valid_labels.cat.rename_categories(np.arange(0,num_labels)) ))

test_dataset = np.float32(np.array(testdata))
# test_labels = testdata.loc[:,'Category']
# test_labels= np.array(pd.get_dummies(test_labels.cat.rename_categories(np.arange(0,num_labels)) ))

print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape)


In [ ]:
N_obs_train

In [ ]:
num_labels

In [ ]:
num_var

In [ ]:
train_labels

In [ ]:
# Adding regularization to the 1 hidden layer network
graph1 = tf.Graph()
batch_size = 128

epoch_size  = N_obs_train / batch_size
num_steps= 2 * epoch_size

import datetime
startTime = datetime.datetime.now() 

def define_and_run_batch(beta):
    
    num_RELU = 1024
    
    with graph1.as_default():
      
      keep_prob = tf.placeholder(tf.float32)
      
      # Input data. For the training data, we use a placeholder that will be fed
      # at run time with a training minibatch.
      tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, num_var)) # remplace batch_size par None
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
      tf_valid_dataset = tf.constant(valid_dataset)
    
      # in Kaggle competitions, we don't know the true labels of the test set
      tf_test_dataset = tf.constant(test_dataset)
  
      # Variables.
      weights_RELU = tf.Variable(
        tf.truncated_normal([num_var, num_RELU],stddev=0.1))
        
      # print(tf.shape(weights_RELU) )   
      #biases_RELU = tf.Variable(tf.zeros([num_RELU]))
      biases_RELU = tf.Variable(tf.random_normal([num_RELU],stddev=0.1))  
        
      weights_layer1 = tf.Variable(
        tf.truncated_normal([num_RELU, num_labels],stddev=0.1))
      #biases_layer1 = tf.Variable(tf.zeros([num_labels]))
      biases_layer1 = tf.Variable(tf.random_normal([num_labels],stddev=0.1))
  
      # Training computation.
      logits_RELU = tf.matmul(tf_train_dataset, weights_RELU) + biases_RELU
      RELU_vec = tf.nn.relu(logits_RELU)
      RELU_vec_drop = tf.nn.dropout(RELU_vec, keep_prob)
    
      logits_layer = tf.matmul(RELU_vec_drop, weights_layer1) + biases_layer1                  
      # loss = tf.reduce_mean(
      #        tf.nn.softmax_cross_entropy_with_logits(logits_layer, tf_train_labels))
      cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits_layer, tf_train_labels,name="cross_entropy")
      l2reg = tf.reduce_sum(tf.square(weights_RELU))+tf.reduce_sum(tf.square(weights_layer1))
      
      loss = tf.reduce_mean(cross_entropy+beta*l2reg)
      logloss = tf.reduce_mean(cross_entropy)
    
  # Optimizer.
      optimizer = tf.train.GradientDescentOptimizer(0.3).minimize(loss)
  
      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(logits_layer)
      
      valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu((tf.matmul(tf_valid_dataset, weights_RELU) + biases_RELU)),weights_layer1)+biases_layer1)
        
      # in Kaggle competitions, we don't know the true labels of the test set     
      test_prediction =tf.nn.softmax(
        tf.matmul(tf.nn.relu((tf.matmul(tf_test_dataset, weights_RELU) + biases_RELU)),weights_layer1)+biases_layer1)
                         
    with tf.Session(graph=graph1) as session:
 
      tf.initialize_all_variables().run()
      print("Initialized")
      for step in range(num_steps):
        
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        # Generate a minibatch. 
        batch_data = train_dataset[offset:(offset + batch_size), :]
        
        batch_labels = train_labels[offset:(offset + batch_size)]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,
                    keep_prob:1.0}
        
        #
        if (step==num_steps-1):
            _, l, predictions_train,predictions_test,predictions_valid = session.run(
              [optimizer, logloss,train_prediction,test_prediction,valid_prediction], feed_dict=feed_dict)
        else:
            _, l, predictions_train = session.run(
              [optimizer, logloss,train_prediction], feed_dict=feed_dict)
            
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions_train, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
      
      # test_acc = accuracy(test_prediction.eval(), test_labels)
      # print("Test accuracy: %.1f%%" % test_acc)

      
        
    x = datetime.datetime.now() - startTime
    print(x)
    return(predictions_test,predictions_valid)
    
    
RES,RES_v,RES_tr=define_and_run_batch(0.001)
N_obs_valid=validdata.shape[0]
logloss_valid=-sum(sum(np.multiply(valid_labels,np.log(RES_v))))/N_obs_valid

print("Validation Logloss =%s" % logloss_valid)


In [ ]:
# 4 layer network

graph1 = tf.Graph()

batch_size = 128

epoch_size  = N_obs_train / batch_size
num_steps= 2 * epoch_size #6501

import datetime
startTime = datetime.datetime.now() 

def define_and_run_batch_4_layer():
    
    

    import math as math

    initial_learning_rate_value = 0.5
    beta1 = 0.001
    beta2 = 0.001
    beta3 = 0.001
    beta4 = 0.001
    beta5 = 0.001

    graph2 = tf.Graph()
    with graph2.as_default():

       # Input data. For the training data, we use a placeholder that will be fed
       # at run time with a training minibatch.
       tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, num_var))
       tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        
       tf_valid_dataset = tf.constant(valid_dataset)
       tf_test_dataset = tf.constant(test_dataset)    
    
       # learning rate decay
       global_step = tf.Variable(0)
       learning_rate = tf.train.exponential_decay(initial_learning_rate_value, global_step, 1, 0.9999)

       # Hidden layer 1
       h1_size = 2048  
       weights_h1 = tf.Variable(tf.truncated_normal([num_var, h1_size], stddev=math.sqrt(2.0/(num_var))))
       biases_h1 = tf.Variable(tf.zeros([h1_size]))
       h1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_h1) + biases_h1)

       # Hidden layer 2
       h2_size = 1024
       weights_h2 = tf.Variable(tf.truncated_normal([h1_size, h2_size], stddev=math.sqrt(2.0/h1_size)))
       biases_h2 = tf.Variable(tf.zeros([h2_size]))
       h2 = tf.nn.relu(tf.matmul(h1, weights_h2) + biases_h2)

       # Hidden layer 3
       h3_size = 512
       weights_h3 = tf.Variable(tf.truncated_normal([h2_size, h3_size], stddev=math.sqrt(2.0/h2_size)))
       biases_h3 = tf.Variable(tf.zeros([h3_size]))
       h3 = tf.nn.relu(tf.matmul(h2, weights_h3) + biases_h3)

       # Hidden layer 4
       h4_size = 256
       weights_h4 = tf.Variable(tf.truncated_normal([h3_size, h4_size], stddev=math.sqrt(2.0/h3_size)))
       biases_h4 = tf.Variable(tf.zeros([h4_size]))
       h4 = tf.nn.relu(tf.matmul(h3, weights_h4) + biases_h4)

       # Output layer
       weights_o = tf.Variable(tf.truncated_normal([h4_size, num_labels], stddev=math.sqrt(2.0/h4_size)))
       biases_o = tf.Variable(tf.zeros([num_labels]))
    
       # Training computation.
       logits = tf.matmul(h4, weights_o) + biases_o
       # loss = tf.reduce_mean(
       #     tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + ((beta1 * tf.nn.l2_loss(weights_h1)) + (beta2 * tf.nn.l2_loss(weights_h2)) 
       #        + (beta3 * tf.nn.l2_loss(weights_h3)) + (beta4 * tf.nn.l2_loss(weights_h4)) + (beta5 * tf.nn.l2_loss(weights_o)))
       
       cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels,name="cross_entropy")
       logloss = tf.reduce_mean(cross_entropy)
    
       loss = logloss + ((beta1 * tf.nn.l2_loss(weights_h1)) + (beta2 * tf.nn.l2_loss(weights_h2)) + (beta3 * tf.nn.l2_loss(weights_h3)) 
                         + (beta4 * tf.nn.l2_loss(weights_h4)) + (beta5 * tf.nn.l2_loss(weights_o)))
       
    
       # Optimizer.
       optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

       # Predictions for the training, validation, and test data.
       train_h1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_h1) + biases_h1)
       train_h2 = tf.nn.relu(tf.matmul(train_h1, weights_h2) + biases_h2)
       train_h3 = tf.nn.relu(tf.matmul(train_h2, weights_h3) + biases_h3)
       train_h4 = tf.nn.relu(tf.matmul(train_h3, weights_h4) + biases_h4)
       train_logits = tf.matmul(train_h4, weights_o) + biases_o
       train_prediction = tf.nn.softmax(train_logits)
  
       valid_h1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_h1) + biases_h1)
       valid_h2 = tf.nn.relu(tf.matmul(valid_h1, weights_h2) + biases_h2)
       valid_h3 = tf.nn.relu(tf.matmul(valid_h2, weights_h3) + biases_h3)
       valid_h4 = tf.nn.relu(tf.matmul(valid_h3, weights_h4) + biases_h4)
       valid_logits = tf.matmul(valid_h4, weights_o) + biases_o
       valid_prediction = tf.nn.softmax(valid_logits)
    
       test_h1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights_h1) + biases_h1)
       test_h2 = tf.nn.relu(tf.matmul(test_h1, weights_h2) + biases_h2)
       test_h3 = tf.nn.relu(tf.matmul(test_h2, weights_h3) + biases_h3)
       test_h4 = tf.nn.relu(tf.matmul(test_h3, weights_h4) + biases_h4)
       test_logits = tf.matmul(test_h4, weights_o) + biases_o
       test_prediction = tf.nn.softmax(test_logits)
  
        
   

# executing the graph     
        
        
    with tf.Session(graph=graph2) as session:
 
      tf.initialize_all_variables().run()
      print("Initialized")
      for step in range(num_steps):
        
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        # Generate a minibatch. 
        batch_data = train_dataset[offset:(offset + batch_size), :]
        
        batch_labels = train_labels[offset:(offset + batch_size)]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        #
        if (step==num_steps-1):
            _, l, predictions_train,predictions_test,predictions_valid = session.run(
              [optimizer, logloss,train_prediction,test_prediction,valid_prediction], feed_dict=feed_dict)
        else:
            _, l, predictions_train = session.run(
              [optimizer, logloss,train_prediction], feed_dict=feed_dict)
            
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions_train, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
      
      # test_acc = accuracy(test_prediction.eval(), test_labels)
      # print("Test accuracy: %.1f%%" % test_acc)
        
    x = datetime.datetime.now() - startTime
    print(x)
    return(predictions_test,predictions_valid)
    
    
    
RES,RES_v=define_and_run_batch_4_layer()
Nobs_valid=validdata.shape[0]
logloss_valid=-sum(sum(np.multiply(valid_labels,np.log(RES_v))))/Nobs_valid
print("Validation Logloss =%s" % logloss_valid)

In [ ]:
# just trying this to see if xgb.cv works
traindata=traindata.loc[0:100000,:]
validdata=validdata.loc[0:50000,:]


In [ ]:
# use sklearn one-hot-encoder only if using sklearn functions
# http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

In [ ]:
###### XGBOOST ######

import datetime
startTime = datetime.datetime.now() 

import xgboost as xgb
data_train   = np.array(traindata.drop('Category',axis=1))
labels_train = np.array(traindata['Category'].cat.codes)

data_valid   = np.array(validdata.drop('Category',axis=1))
labels_valid = np.array(validdata['Category'].astype('category').cat.codes)

weights_train = np.ones(len(labels_train))
weights_valid  = np.ones(len(labels_valid ))

dtrain = xgb.DMatrix( data_train, label=labels_train,weight = weights_train)
dvalid  = xgb.DMatrix( data_valid , label=labels_valid ,weight = weights_valid )




param_train = {'bst:max_depth':5, 'bst:eta':0.3, # eta [default=0.3]
         #'min_child_weight':1,'gamma':0,'subsample':1,'colsample_bytree':1,'scale_pos_weight':0, # default
         # max_delta_step:0 # default
         'min_child_weight':5,'scale_pos_weight':0, 'max_delta_step':2,'lambda':2,
         'subsample':0.8,'colsample_bytree':0.8,
         'silent':1, 'objective':'multi:softprob','eval_metric':'mlogloss','num_class':39,'nthread':4,
         'evals_result':['eval_metric'],'evallist': [(dtrain,'train'),(dvalid,'eval')]}

param_CV = {'bst:max_depth':5, 'bst:eta':0.3,        
         'min_child_weight':5,'scale_pos_weight':0, 'max_delta_step':2,'lambda':2,
         'subsample':0.8,'colsample_bytree':0.8,
         'silent':1, 'objective':'multi:softprob','eval_metric':'mlogloss','num_class':39,'nthread':4,
         'evals_result':['eval_metric'],
         'verbose_eval':True}

evallist  = [(dtrain,'train'),(dvalid,'eval')] # if there is a validation set
# evallist  = [(dtrain,'train')]                   # if there is no validation set

num_round = 100

bst = xgb.train( param_train, dtrain, num_round, evallist,early_stopping_rounds=5 ) # early_stopping_rounds=10 # when there is a validation set

res=xgb.cv(param_CV,dtrain=dtrain,num_boost_round=num_round,nfold = 5,early_stopping_rounds=5)

# bst.save_model('0001.model')

# dump model
# bst.dump_model('dump.raw.txt')
# dump model with feature map
# bst.dump_model('dump.raw.txt','featmap.txt')

x = datetime.datetime.now() - startTime
print(x)

Will train until eval error hasn't decreased in 5 rounds.
[0]	train-mlogloss:3.237984	eval-mlogloss:3.310841
[1]	train-mlogloss:2.950211	eval-mlogloss:3.066134


In [ ]:
res

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
xgb.plot_importance(bst)
#xgb.plot_tree(bst, num_trees=2)
#xgb.to_graphviz(bst, num_trees=2)

In [ ]:
data_test   = np.array(df_t)

dtest = xgb.DMatrix( data_test)

In [ ]:
predXGB = bst.predict(dtest)


In [ ]:
df_submit=pd.read_csv('/Users/francois-guillaume.rideau/Documents/Kaggle/Kaggle-SF/sampleSubmission.csv',dtype=float)
df_submit.head()

In [ ]:
df_submit.shape
type(df_submit)

In [ ]:
for i in range (1,40): 
    # df_submit[df_submit.columns[i]]=predXGB[:,i-1] # for XGBoost
    df_submit[df_submit.columns[i]]=RES[:,i-1]   # for TensorFlow
df_submit['Id']=df_submit['Id'].astype(int)



In [ ]:
df_submit.head()

In [ ]:
filename = '/Users/francois-guillaume.rideau/Documents/Kaggle/Kaggle-SF/submitTF4.csv'
df_submit.to_csv(filename,index=False)

In [ ]:
df_verify=pd.read_csv(filename,dtype=float)
df_verify.head()

In [ ]:
z=df_verify.describe()
z

In [ ]:
# checking if the mean of predicted probabilities match with the distribution in the training set
zz=z.loc['mean',:].sort_values(ascending=False)
zz

In [ ]:
# distribution of Category in the training set
df_raw['Category'].value_counts()/877982

In [ ]:
predXGB_train = bst.predict(dtrain)


In [ ]:
predXGB_train=pd.DataFrame(predXGB_train)
predXGB_train

In [ ]:
predXGB_train.columns=df_raw['Category'].cat.categories

In [ ]:
z=predXGB_train.describe()